# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import scipy.stats as st

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

## Загрузка и подготовка данных

In [2]:
try:
    gd0 = pd.read_csv(r'C:\Users\vk\data_science\data\regions\geo_data_0.csv')
    gd1 = pd.read_csv(r'C:\Users\vk\data_science\data\regions\geo_data_1.csv')
    gd2 = pd.read_csv(r'C:\Users\vk\data_science\data\regions\geo_data_2.csv')
except:
    gd0 = pd.read_csv(r'/datasets/geo_data_0.csv')
    gd1 = pd.read_csv('/datasets/geo_data_1.csv')
    gd2 = pd.read_csv('/datasets/geo_data_2.csv')

gd0.info()
print()
gd1.info()
print()
gd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Nul

- Загружаем данные, выводим информацию по кажому `DataFrame` региона

In [3]:
print(gd0.head())
print()
print(gd1.head())
print()
print(gd2.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


- Выводим первые пять строк каждого `DataFrame` региона

In [4]:
print("Сумма дубликатов в geo_data_0:", gd0.duplicated().sum())
print()
print("Сумма дубликатов в geo_data_1:", gd1.duplicated().sum())
print()
print("Сумма дубликатов в geo_data_2:", gd2.duplicated().sum())

Сумма дубликатов в geo_data_0: 0

Сумма дубликатов в geo_data_1: 0

Сумма дубликатов в geo_data_2: 0


- Выводим кол-во дубликатов в `DataFrame` каждого региона

In [5]:
print(gd0.describe())
print()
print(gd1.describe())
print()
print(gd2.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%      

- Выводим описательную статистику по каждому региону

## Обучение и проверка модели

In [6]:
def split(data):
    features = data.drop(['product','id'], axis=1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25,
                                                                                  random_state=17)
    pd.options.mode.chained_assignment = None
    scaler = MinMaxScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    lr = LinearRegression(normalize=False)
    lr.fit(features_train, target_train)
    predictions = lr.predict(features_valid)
    predictions = pd.Series(predictions)
    
    rmse = (mean_squared_error(predictions, target_valid))**(0.5)
    average_product = sum(predictions) / len(predictions)
    
    print('RMSE: {0:.2f}'.format(rmse))
    print('Кол-во нефти в среднем: {0:.2f}'.format(average_product))
    return (predictions, target_valid.reset_index(drop=True), rmse)

Пишем функцию `split` которая:
1. Делит выборку на валидационнцю и обучающую
2. Скалирует данные
3. Обучает модель на основе подготовленных данных
4. Высчитывает `predictions`
5. Находит `RMSE`

In [7]:
print('geo_data_0')
predict_0, valid_0, rmse_0 = split(gd0)
print()

print('geo_data_1')
predict_1, valid_1, rmse_1 = split(gd1)

print()
print('geo_data_2')
predict_2, valid_2, rmse_2 = split(gd2)

geo_data_0
RMSE: 37.79
Кол-во нефти в среднем: 92.39

geo_data_1
RMSE: 0.89
Кол-во нефти в среднем: 68.82

geo_data_2
RMSE: 40.14
Кол-во нефти в среднем: 95.09


- Записываем полученные данные в переменные

## Подготовка к расчёту прибыли

In [8]:
total_budget = 10**10 
income_per_barrel = 450 * 10**3

break_even = total_budget / income_per_barrel
print('Точка безубыточности:', round(break_even), 'тыс. баррелей')
print('Средний объем нефти для 1 скважины для выхода на ТБ:', np.ceil(break_even/200))

Точка безубыточности: 22222 тыс. баррелей
Средний объем нефти для 1 скважины для выхода на ТБ: 112.0


- Задаем переменные и присваиваем им значения
- Высчитываем ТБ(точку безубыточности)

## Расчёт прибыли и рисков 

In [9]:
def profit_func(predictions, target):
    top_predictions = predictions.sort_values(ascending=False)
    top_target = target[top_predictions.index][:200]
    revenue = top_target.sum() * income_per_barrel
    return revenue - total_budget

- Пишем функцию которая расчитывает прибыль

In [10]:
state = np.random.RandomState(17)

In [11]:
def confidence_risk(predictions, target):
    revenue = []
    
    for i in range(1000):
        target_sample = target.sample(500, replace=True, random_state=state)
        predictions_sample = predictions[target_sample.index]
        revenue.append(profit_func(predictions_sample, target_sample))
        
    lower = int(np.percentile(revenue, 2.5))
    higher = int(np.percentile(revenue, 97.5))
    mean_revenue = int(sum(revenue) / len(revenue))
    risk = st.percentileofscore(revenue, 0)

    return ((lower, higher), mean_revenue, risk)

- Пишем функцию которая расчитывае доверительный интервал и риск

In [12]:
%%time
i = 0
for predictions, targets in zip([predict_0, predict_1, predict_2], 
                         [valid_0, valid_1, valid_2]):
    interval, mean_revenue, risk = confidence_risk(predictions, targets)
    print('Регион: geo data', i)
    i += 1
    print('Среднее:', mean_revenue)
    print('95% доверительный интервал:', interval)
    print('Риск:', risk,'%')
    print()

Регион: geo data 0
Среднее: 471268306
95% доверительный интервал: (-65729586, 986488701)
Риск: 4.6000000000000005 %

Регион: geo data 1
Среднее: 494222368
95% доверительный интервал: (77802011, 913637149)
Риск: 1.1 %

Регион: geo data 2
Среднее: 352922017
95% доверительный интервал: (-187223101, 876785878)
Риск: 10.600000000000001 %

Wall time: 5.23 s


- Автоматизируем присваивание функции по регионам, чтобы было быстрее посчитать все.

---

## Общий вывод

- В самом начале мы подготовили данные для анализа, изучили их типы, проверили на наличие дубликатов, вывели описательную статистику по каждому региону.
- Затем была написана функция, которая разбивает данные на выборки, скалирует их, строит модель линейной регрессии для того, что предсказать объемы нефти в каждой из скважин.
- Далее, с помощью `Bootstrap` был найден 95% доверительный интервал. 
- Наименее рискованным является второй регион, где риск составил 1.1%, средняя прибыль также является наибольшей для второго региона.
- С точки зрения перспектив разработки, наиболее привлекательным является второй регион.